In [140]:
import imaplib
import email
from email.header import decode_header
import pandas as pd
import time  


In [165]:
usuario = ""
contraseña = ""

In [167]:

# Conectar al servidor de Gmail usando IMAP
servidor = "imap.gmail.com"
puerto = 993
mail = imaplib.IMAP4_SSL(servidor, puerto)
mail.login(usuario, contraseña)

('OK', [b'authenticated (Success)'])

In [168]:
# Listar todas las carpetas en la cuenta
estado, carpetas = mail.list()
for carpeta in carpetas:
    print(carpeta.decode())

(\HasNoChildren) "/" "INBOX"
(\HasNoChildren) "/" "Junk"
(\HasNoChildren) "/" "Sent"
(\HasNoChildren) "/" "Trash"
(\HasChildren \Noselect) "/" "[Gmail]"
(\Drafts \HasNoChildren) "/" "[Gmail]/Borradores"
(\Flagged \HasNoChildren) "/" "[Gmail]/Destacados"
(\HasNoChildren \Sent) "/" "[Gmail]/Enviados"
(\HasNoChildren \Important) "/" "[Gmail]/Importantes"
(\HasNoChildren \Trash) "/" "[Gmail]/Papelera"
(\HasNoChildren \Junk) "/" "[Gmail]/Spam"
(\All \HasNoChildren) "/" "[Gmail]/Todos"


In [171]:
def obtener_correos(carpeta, etiqueta, limite=1000, fragmento=100):
    
    # Seleccionar la carpeta
    estado, _ = mail.select(carpeta)
    if estado != "OK":
        print(f"Error al seleccionar la carpeta {carpeta}. Estado: {estado}")
        return []

    print(f"Carpeta {carpeta} seleccionada correctamente.")

    # Buscar todos los correos en la carpeta
    estado, mensajes = mail.search(None, "ALL")
    if estado != "OK":
        print(f"Error en la búsqueda de correos en la carpeta {carpeta}")
        return []

    # Limitar la cantidad de correos
    id_mensajes = mensajes[0].split()[:limite]
    correos = []
    
    # Procesar en fragmentos
    for i in range(0, len(id_mensajes), fragmento):
        fragmento_ids = id_mensajes[i:i + fragmento]
        id_list = ",".join(fragmento_id.decode("utf-8") for fragmento_id in fragmento_ids)  # Corrección aquí
        
        estado, data = mail.fetch(id_list, "(RFC822)")
        if estado != "OK":
            print(f"Error al obtener el fragmento {i // fragmento + 1}")
            continue

        for response_part in data:
            if isinstance(response_part, tuple):
                mensaje = email.message_from_bytes(response_part[1])

                # Decodificar el asunto (coloca aquí el ajuste)
                asunto, codificacion = decode_header(mensaje["Subject"] or "")[0]
                if isinstance(asunto, bytes):
                    try:
                        asunto = asunto.decode(codificacion if codificacion else "utf-8")
                    except LookupError:
                        asunto = asunto.decode("utf-8", errors="replace")

                # Decodificar el remitente
                de, codificacion = decode_header(mensaje.get("From"))[0]
                if isinstance(de, bytes):
                    try:
                        de = de.decode(codificacion if codificacion else "utf-8")
                    except LookupError:
                        de = de.decode("utf-8", errors="replace")

                # Obtener el cuerpo
                cuerpo = ""
                if mensaje.is_multipart():
                    for parte in mensaje.walk():
                        tipo_contenido = parte.get_content_type()
                        disp = str(parte.get("Content-Disposition"))
                        if tipo_contenido == "text/plain" and "attachment" not in disp:
                            cuerpo = parte.get_payload(decode=True).decode(errors="replace")
                            break
                else:
                    cuerpo = mensaje.get_payload(decode=True).decode(errors="replace")

                # Añadir correo al listado
                correos.append({
                    "Asunto": asunto,
                    "Remitente": de,
                    "Cuerpo": cuerpo,
                    "Etiqueta": etiqueta
                })
        
        print(f"Fragmento {i // fragmento + 1} procesado. Total de correos hasta ahora: {len(correos)}")
        time.sleep(1)  # Añadir una pausa para evitar límites de conexión

    print(f"Total de correos procesados en {carpeta}: {len(correos)}")
    return correos


In [ ]:

df_correos.to_csv("correos_etiquetados_spam.csv", index=False)


In [ ]:
# Obtener correos de Spam y de Importante
correos_spam = obtener_correos("[Gmail]/Spam", "spam", limite=1000)
correos_importante = obtener_correos("[Gmail]/Importantes", "importante", limite=1000)
correos_spam2 = obtener_correos("Trash", "spam", limite=1000)

# Combinar ambas listas en un solo DataFrame
df_correos = pd.DataFrame(correos_spam2 + correos_spam + correos_importante)

# Mostrar el DataFrame (o puedes guardarlo en un archivo)
print(df_correos.head())

# O guardar en un archivo CSV
df_correos.to_csv("correos_etiquetados.csv", index=False)


Carpeta [Gmail]/Spam seleccionada correctamente.
Fragmento 1 procesado. Total de correos hasta ahora: 25
Total de correos procesados en [Gmail]/Spam: 25
Carpeta [Gmail]/Importantes seleccionada correctamente.
Fragmento 1 procesado. Total de correos hasta ahora: 50
